In [1]:
import numpy as np
import tensorflow as tf

2025-07-20 06:05:26.150342: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752991526.192832   71972 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752991526.208170   71972 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752991526.253539   71972 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752991526.253599   71972 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752991526.253603   71972 computation_placer.cc:177] computation placer alr

In [2]:
import keras
wte = keras.layers.Embedding(input_dim=10, output_dim=3)

In [3]:
wte(np.array([1, 2, 3, 4, 5]))  # Example usage

I0000 00:00:1752991565.181845   71972 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2242 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5


<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[-0.02860935,  0.00974352,  0.04084095],
       [-0.04977075, -0.03769188, -0.03748412],
       [-0.01952647, -0.01114391,  0.02047164],
       [-0.0404057 , -0.03201417, -0.04534901],
       [ 0.01393386, -0.04424917, -0.01094481]], dtype=float32)>

In [6]:
wte.weights, wte.embeddings  # Accessing weights, kernel, and embeddings

([<Variable path=embedding/embeddings, shape=(10, 3), dtype=float32, value=[[ 1.02963559e-02  4.80997562e-03  1.83694065e-05]
   [-2.86093485e-02  9.74352285e-03  4.08409499e-02]
   [-4.97707501e-02 -3.76918800e-02 -3.74841206e-02]
   [-1.95264705e-02 -1.11439116e-02  2.04716437e-02]
   [-4.04057018e-02 -3.20141688e-02 -4.53490131e-02]
   [ 1.39338635e-02 -4.42491658e-02 -1.09448060e-02]
   [-2.34466791e-02 -3.12236678e-02 -2.88974494e-04]
   [-2.81121023e-02 -4.56346646e-02  3.39681022e-02]
   [-3.66366282e-02  8.73392820e-03  2.19239853e-02]
   [-7.10852072e-03  4.43816446e-02 -2.46741623e-03]]>],
 <Variable path=embedding/embeddings, shape=(10, 3), dtype=float32, value=[[ 1.02963559e-02  4.80997562e-03  1.83694065e-05]
  [-2.86093485e-02  9.74352285e-03  4.08409499e-02]
  [-4.97707501e-02 -3.76918800e-02 -3.74841206e-02]
  [-1.95264705e-02 -1.11439116e-02  2.04716437e-02]
  [-4.04057018e-02 -3.20141688e-02 -4.53490131e-02]
  [ 1.39338635e-02 -4.42491658e-02 -1.09448060e-02]
  [-2.34

In [8]:
r = tf.random.uniform(shape=(10, 3), dtype=tf.bfloat16)
print(r, r.dtype)  # Example of random tensor with bfloat16 dtype
r2 = r * r
print(r2, r2.dtype)  # Example of element-wise multiplication with bfloat16 dtype


tf.Tensor(
[[0.8125 0.0625 0.328125]
 [0.859375 0.5625 0.84375]
 [0.429688 0.3125 0.0234375]
 [0.265625 0.335938 0.273438]
 [0.0859375 0.875 0.742188]
 [0.34375 0.570312 0.898438]
 [0.523438 0.484375 0.742188]
 [0.148438 0.359375 0]
 [0.320312 0.289062 0.484375]
 [0.6875 0.3125 0.53125]], shape=(10, 3), dtype=bfloat16) <dtype: 'bfloat16'>
tf.Tensor(
[[0.660156 0.00390625 0.107422]
 [0.738281 0.316406 0.710938]
 [0.18457 0.0976562 0.000549316]
 [0.0703125 0.112793 0.074707]
 [0.00738525 0.765625 0.550781]
 [0.118164 0.326172 0.808594]
 [0.273438 0.234375 0.550781]
 [0.0219727 0.128906 0]
 [0.102539 0.0834961 0.234375]
 [0.472656 0.0976562 0.28125]], shape=(10, 3), dtype=bfloat16) <dtype: 'bfloat16'>


In [ ]:
def get_angles(pos, i, d_model):
        angle_rates = 1 / np.power(160000, (2 * (i // 2)) / np.float32(d_model))
        return pos * angle_rates

def positional_encoding(position, d_model):
    angle_rads = get_angles(
        np.arange(position)[:, np.newaxis],
        np.arange(d_model)[np.newaxis, :],
        d_model,
    )

    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)

In [24]:
import random

d_model = 512
position = random.randint(1, 160000)
pos_encoding = positional_encoding(position, d_model)
print(f"Positional Encoding for position {position} and d_model {d_model}:\n{pos_encoding.numpy()}")
print(f"Shape: {pos_encoding.shape}")
print(f"Data type: {pos_encoding.dtype}")
print(f"Size in bytes: {pos_encoding.numpy().nbytes}")
print(f"Size in MB: {pos_encoding.numpy().nbytes / (1024 * 1024)} MB")

Positional Encoding for position 33329 and d_model 512:
[[[ 0.0000000e+00  1.0000000e+00  0.0000000e+00 ...  1.0000000e+00
    0.0000000e+00  1.0000000e+00]
  [ 8.4147096e-01  5.4030228e-01  8.1589204e-01 ...  1.0000000e+00
    6.5495069e-06  1.0000000e+00]
  [ 9.0929741e-01 -4.1614684e-01  9.4350451e-01 ...  1.0000000e+00
    1.3099014e-05  1.0000000e+00]
  ...
  [-1.4868733e-02  9.9988943e-01  3.2450861e-01 ...  9.7395545e-01
    2.1653989e-01  9.7627378e-01]
  [ 8.3334434e-01  5.5275416e-01 -5.8410597e-01 ...  9.7395390e-01
    2.1654628e-01  9.7627234e-01]
  [ 9.1538447e-01 -4.0258074e-01 -9.9997371e-01 ...  9.7395235e-01
    2.1655267e-01  9.7627091e-01]]]
Shape: (1, 33329, 512)
Data type: <dtype: 'float32'>
Size in bytes: 68257792
Size in MB: 65.095703125 MB


In [8]:
class Patches(tf.keras.layers.Layer):
    def __init__(self, patch_size, strides_format="independent"):
        super(Patches, self).__init__()
        self.patch_size = patch_size
        if strides_format == "independent":
            self.strides = [1, self.patch_size, self.patch_size, 1]
        elif strides_format == "overlapping":
            self.strides = [1, self.patch_size // 2, self.patch_size // 2, 1]

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=self.strides,
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        print(f"Extracted patches shape: {patches.shape}")
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        result = np.zeros(shape=(patches.shape[0], patches.shape[1] + 2, patches.shape[2]))
        result[:, 1:-1, :] = patches
        result[:, -1, :] = 1
        return tf.convert_to_tensor(result)

In [9]:
layer = Patches(16)
image = tf.random.uniform((1, 128, 128, 3), dtype=tf.float32)
patches = layer(image)
print(f"Patches shape: {patches.shape}")

Extracted patches shape: (1, 8, 8, 768)
Patches shape: (1, 66, 768)
